In [1]:
import pandas as pd
import numpy as np
import datetime
import psycopg2
from psycopg2.extras import Json
import json
import requests
from datetime import date

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
# Automate the execution of this to run every day until Feb 20th, 2024 when the season has finished
# After 2023 season has ended change the year_list variable below to be 2024

year = 2023

In [4]:
# Retrieve postgres database credentials from json file
with open('nfl_project_postgres_info.json', 'r') as file:
    # Read the file contents
    postgres_json_data = file.read()

    # Parse the JSON data
    postgres_info = json.loads(postgres_json_data)

In [5]:
# Retrieve postgres database credentials from json file
with open('rapid_api_weather.json', 'r') as file:
    # Read the file contents
    weather_api_json_data = file.read()

    # Parse the JSON data
    weather_api_info = json.loads(weather_api_json_data)

In [6]:
# Connect to the PostgreSQL database
conn = psycopg2.connect(
    
        host="localhost",
        database=postgres_info['database_name'],
        user=postgres_info['nfl_project_username'],
        password=postgres_info['nfl_project_password']
)

# Create a cursor
cursor = conn.cursor()

# Prepare the SQL statement to delete records
sql = "SELECT espn_game_id, game_location, game_time, game_date, weather FROM dbo.nfl_game_data WHERE season = %s"

# Execute the delete statement
cursor.execute(sql, (year,))

# Fetch the results
results = cursor.fetchall()

# Get the column names from the cursor description
columns = [desc[0] for desc in cursor.description]

# Create a DataFrame from the results and column names
game_table = pd.DataFrame(results, columns=columns)

# Close the cursor and connection
cursor.close()
conn.close()

In [7]:
# Sort the game_table
game_table = game_table.sort_values(by=["game_date", "game_time"]).reset_index(drop=True)

In [8]:
# Query weather api by looping through the game_table
url = "https://weatherapi-com.p.rapidapi.com/current.json"
headers = {
    "X-RapidAPI-Key": weather_api_info['api_key'],
    "X-RapidAPI-Host": "weatherapi-com.p.rapidapi.com"
}


results = {}  # Dictionary to store the results

for index, row in game_table.iterrows():
    
    try:
        city = row['game_location']
        hour = row['game_time'].hour
        
    except AttributeError: continue
    
    querystring = {
        "q": city,
        "dt": date.today(),
        "lang": "en",
        "hour": hour
    }
    
    response = requests.get(url, headers=headers, params=querystring)
    data = response.json()
    
    # Store the weather data in the results dictionary
    results[index] = data

In [9]:
# Update the DataFrame with the weather information
game_table['weather'] = pd.Series(results)

In [10]:
# Drop records with NaN values in any column
game_table = game_table.dropna().reset_index(drop=True)

In [11]:
# Extract only desired bits of information from weather results
game_table['weather'] = game_table['weather'].apply(lambda x: {'temp_f': x['current']['temp_f'],
                                                               'wind_mph': x['current']['wind_mph'],
                                                               'pressure_in': x['current']['pressure_in'],
                                                               'precip_in': x['current']['precip_in'],
                                                               'humidity': x['current']['humidity'],
                                                               'cloud': x['current']['cloud'],
                                                               'vis_miles': x['current']['vis_miles'],
                                                               'gust_mph': x['current']['gust_mph']} if isinstance(x, dict) else x)

In [12]:
# Convert the "weather" column to string representation
game_table['weather'] = game_table['weather'].apply(str)

In [16]:
# Connect to the PostgreSQL database
conn = psycopg2.connect(
    
        host="localhost",
        database=postgres_info['database_name'],
        user=postgres_info['nfl_project_username'],
        password=postgres_info['nfl_project_password']
)

# Create a cursor
cursor = conn.cursor()

# Iterate over the DataFrame rows
for _, row in game_table.iterrows():
    espn_game_id = row['espn_game_id']
    weather_text = row['weather']
    
    # Prepare the SQL statement to update the "weather" column
    sql = """
    UPDATE dbo.nfl_game_data
    SET weather = %s
    WHERE espn_game_id = %s
    """
    
    # Execute the SQL statement with the weather data and espn_game_id as parameters
    cursor.execute(sql, (weather_text, espn_game_id))

# Commit the changes
conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()